In [1]:
import numpy as np

In [2]:
import os
import torch
from torch.utils.data import DataLoader, random_split
from lightning.pytorch import Trainer

/home/demancum/miniconda3/envs/cocola/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
os.environ['CUDA_VISIBLE_DEVICES'] = '1'  

In [4]:
from contrastive_model.contrastive_model import CoCola
from data import (
musdb_contrastive_preprocessed,
moisesdb_contrastive_preprocessed,
slakh2100_contrastive_preprocessed,
coco_chorales_contrastive_preprocessed
)


In [5]:
#CHECKPOINT = '/speech/dbwork/mul/spielwiese4/students/demancum/cocola/k5llcdm8/checkpoints/epoch=21-step=21406.ckpt' #bumbling-breeze-34 bs=512 chunk_dur=2s
#CHECKPOINT = '/speech/dbwork/mul/spielwiese4/students/demancum/cocola/r71kxj7r/checkpoints/epoch=36-step=29119.ckpt' #good-wave-24 bs=256 chunk_dur=5s
#CHECKPOINT = '/speech/dbwork/mul/spielwiese4/students/demancum/cocola/shov44gs/checkpoints/epoch=24-step=19900.ckpt' #golden-dew-25 bs=128 chunk_dur=10s

#CHECKPOINT = '/speech/dbwork/mul/spielwiese4/students/demancum/cocola/zr304efn/checkpoints/epoch=23-step=17448.ckpt' #flowing-sound-26 bs=96 chunk_dur=15s

#CHECKPOINT = '/speech/dbwork/mul/spielwiese4/students/demancum/cocola/8jtiowyz/checkpoints/epoch=46-step=38963.ckpt' #jumping-breeze-27 bs=64 chunk_dur=20s

#CHECKPOINT = '/speech/dbwork/mul/spielwiese3/demancum/cocola/e3xubuzt/checkpoints/epoch=38-step=245622.ckpt' #frosty-silence-39 musdb 32bs test

CHECKPOINT = '/speech/dbwork/mul/spielwiese3/demancum/cocola/2w5qgehd/checkpoints/checkpoint-valid_loss=0.8805-epoch=21.ckpt' #sweet-cosmos-41 more_negative_NO_HPSS

model = CoCola.load_from_checkpoint(CHECKPOINT)
trainer = Trainer()


/home/demancum/miniconda3/envs/cocola/lib/python3.11/site-packages/lightning/fabric/plugins/environments/slurm.py:204: The `srun` command is available on your system but is not used. HINT: If your intention is to run Lightning on SLURM, prepend your python command with `srun` like so: srun python /home/demancum/miniconda3/envs/cocola/lib/python3.11 ...
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs


## MOISESDB

In [8]:
dataset = moisesdb_contrastive_preprocessed.get_dataset(
    chunk_duration=5,
    positive_noise=0,
    generate_submixtures=True,
    transform=None
    )

_, _, test_dataset = random_split(
    dataset=dataset,
    lengths=[0.8, 0.1, 0.1],
    generator=torch.Generator().manual_seed(42)
    )

test_dataloader = DataLoader(
    test_dataset,
    batch_size=64,
    shuffle=False,
    drop_last=True,
    num_workers=os.cpu_count(),
    persistent_workers=True
    )


INFO:Found original dataset at /disk1/demancum/moisesdb/moisesdb_v0.1.
INFO:Found preprocessed dataset at /disk1/demancum/moisesdb/preprocessed5.


In [9]:
trainer.test(model=model, dataloaders=test_dataloader)


LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [1]


Testing DataLoader 0: 100%|██████████| 32/32 [00:02<00:00, 15.08it/s]


┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃        Test metric        ┃       DataLoader 0        ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│       test_accuracy       │       0.39404296875       │
│         test_loss         │    3.0276100635528564     │
└───────────────────────────┴───────────────────────────┘

[{'test_loss': 3.0276100635528564, 'test_accuracy': 0.39404296875}]

## slakh2100

In [11]:
test_dataset = slakh2100_contrastive_preprocessed.get_dataset(
    split="test",
    chunk_duration=5,
    positive_noise=0,
    generate_submixtures=True,
    transform=None
    )

test_dataloader = DataLoader(
    test_dataset,
    batch_size=2,
    shuffle=False,
    drop_last=True,
    num_workers=os.cpu_count(),
    persistent_workers=True
    )

trainer.test(model=model, dataloaders=test_dataloader)

INFO:Found original dataset split test at /disk1/demancum/slakh2100_contrastive/original/slakh2100_redux_16k/test.
INFO:Found preprocessed dataset split test at /disk1/demancum/slakh2100_contrastive/preprocessed5/test.
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [1]


Testing DataLoader 0: 100%|██████████| 7946/7946 [01:59<00:00, 66.54it/s]


┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃        Test metric        ┃       DataLoader 0        ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│       test_accuracy       │    0.9180719859048578     │
│         test_loss         │    0.20061495900154114    │
└───────────────────────────┴───────────────────────────┘

[{'test_loss': 0.20061495900154114, 'test_accuracy': 0.9180719859048578}]

## COCOCHORALES

In [14]:
test_dataset = coco_chorales_contrastive_preprocessed.get_dataset(
    split="test",
    ensemble="*",
    chunk_duration=5,
    positive_noise=0,
    generate_submixtures=True,
    transform=None
    )
    
test_dataloader = DataLoader(
    test_dataset,
    batch_size=2,
    shuffle=False,
    drop_last=True,
    num_workers=os.cpu_count(),
    persistent_workers=True
    )


INFO:Found original dataset split test at /disk1/demancum/coco_chorales_contrastive/original/test.


INFO:Found preprocessed dataset split test at /disk1/demancum/coco_chorales_contrastive/preprocessed5/test.


In [15]:
trainer.test(model=model, dataloaders=test_dataloader)

LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [1]


Testing DataLoader 0: 100%|██████████| 9338/9338 [02:17<00:00, 67.68it/s]


┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃        Test metric        ┃       DataLoader 0        ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│       test_accuracy       │    0.9957164275005355     │
│         test_loss         │   0.013525441288948059    │
└───────────────────────────┴───────────────────────────┘

[{'test_loss': 0.013525441288948059, 'test_accuracy': 0.9957164275005355}]

## MusDB


In [17]:
# Using train split since it is not used from training
test_dataset = musdb_contrastive_preprocessed.get_dataset(
    split="train",
    chunk_duration=5,
    positive_noise=0,
    generate_submixtures=True,
    transform=None
    )

test_dataloader = DataLoader(
    test_dataset,
    batch_size=2,
    shuffle=False,
    drop_last=True,
    num_workers=os.cpu_count(),
    persistent_workers=True
    )

trainer.test(model=model, dataloaders=test_dataloader)



INFO:Found original dataset split train at /disk1/demancum/musdb_contrastive/original/train.
INFO:Found preprocessed dataset split train at /disk1/demancum/musdb_contrastive/preprocessed5/train.
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [1]


Testing DataLoader 0: 100%|██████████| 4619/4619 [01:11<00:00, 64.85it/s]


┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃        Test metric        ┃       DataLoader 0        ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│       test_accuracy       │    0.8844988092660748     │
│         test_loss         │    0.42330029606819153    │
└───────────────────────────┴───────────────────────────┘

[{'test_loss': 0.42330029606819153, 'test_accuracy': 0.8844988092660748}]